Group:
* Minjie Zhu
* Hanchao Lei
* Anass Meskini

Define: $L(t) = L(t, T - \Delta,T)$

We want to price the contract paying at $T$:
$$\max \big[0,\left(\frac{S(T)}{S(0)} - k \right) \left(\frac{L(T)}{L(0)}-k' \right)\big]$$

# Section 1: LIBOR market model

Since  $B^fX$ is a traded asset, $\frac{B^fX}{B^d}$ is a $Q^d$-martingale and:

$$\frac{dX}{X} = (r_d - r_f) dt + \sigma_X dW_X^d(t)$$
We work under the measure $Q^T$ and assume the dynamics for LIBOR:
$$dL(t) = L(t)\sigma_L(t) dW^T_L(t)$$

We also have:
$$\frac{dS}{S} = (r_f - \sigma_S\sigma_X\rho_{SX} + \sigma_S\sigma_T\rho_{ST}) dt + \sigma_SdW^T_S(t)$$

With:

$$<dW_X^d, dW_S^d> = \rho_{SX}dt$$
$$<dW_S^d, dW_T^d> = \rho_{ST}dt$$
$$<dW_S^d, dW_L^d> = \rho_{SL}dt$$
If we assume that the correlations are constant, we have a closed form for both processes under $Q^T$:

\begin{aligned}
& S(T) = S(0) e^{(\mu_S - \frac{\sigma_S^2}{2})T + \sigma_S W^S_T(t)} \\ 
& L(T-\Delta) = L(0) e^{ -\frac{\sigma_L^2}{2}(T-\Delta) + \sigma_L W^T_L (T-\Delta)}
\end{aligned}

And we can price the option with a one step Monte-Carlo simulation.

## Price an option

In [18]:
import numpy as np

# market info
rf = .01     # risk-free rate
volS = .05   # volatility of S
volX = .05   # volatility of X 
volT = .01   # volatility of p(t,T)
volL = .05   # volatility of L(t,T)
corrSX = 0.0  # correlation of S, X
corrST = 0.0  # correlation of S, p(t,T)
corrSL = 0.3  # correlation of S, L
p_0_T = 0.75  # p(0, T)

# option parameters
stock_strike = 2.   # k
libor_strike = 1.5  # k'
T = 20      # maturity
delta_T = 2 #

# MC parameters
Nsim = 100000


val = .0
for i in range(Nsim):
    # generate two correlated normals
    z0 = np.random.normal()
    z1 = corrSL * z0 + np.sqrt(1 - corrSL**2) * np.random.normal()
    
    # generate the stock price
    drift = rf + volS*volX*corrSX + volS*volT*corrST
    stock_ratio = np.exp((drift -  0.5 * volS**2 ) * T + volS * np.sqrt(T) * z0)
    
    # generate the LIBOR
    libor_ratio = np.exp((-0.5 * volL **2) * (T-delta_T) + volL * np.sqrt(T-delta_T) * z1)
    
    val += max((stock_ratio - stock_strike) * (libor_ratio - libor_strike), 0.)

print("Option value: {:.2f}".format(p_0_T * val / Nsim))

Option value: 0.31


## Hetson model

We now relax the assumption of a constant stock volatitlity and use a stochastic volatility model specifically the Hetson model. Under real world probabilities, we have:

$$\frac{dS}{S} = \mu dt + \sqrt{v_t}dW^P_1(t)$$
$$dv_t = k(v_{0} - v_t)dt + \xi  c dW^P_2$$

With $<dW^P_1, dW^P_2> = \rho_{SV}dt$. $S$ is traded so under foreign risk-neutral measure, we have: 
$$\frac{dS}{S} = r_fdt + \sqrt{v_t}dW^f_1(t)$$

$v_t$ is not traded. If $\lambda_t$ is its market price of risk, we have:
$$dv_t = k(v_{0} - v_t - \lambda_t \xi \sqrt{v_t})dt + \xi  \sqrt{v_t} dW^f_2$$

Under the $Q^T$ measure we have:
$$\frac{dS}{S} = (r_f + \beta_0 \sqrt{v_t}) dt + \sqrt{v_t}dW^T_S(t)$$

$$dv_t = k(v_{0} - v_t + \beta_1 \sqrt{v_t})dt + \xi  \sqrt{v_t} dW^T_v$$

with: $\beta_0 = \sigma_T\rho_{ST} - \sigma_X\rho_{SX}$
and $\beta_1 = \xi (\sigma_T\rho_{VT} - \sigma_X\rho_{VX} - \lambda)$ if we choose a constant market price of risk for volatility linked contracts.

The Feller condition is:
$2kv_0 > \xi^2$

In [19]:
# Hetson model parameters
k = 0.2     # rate of reversion
v0 = volS   # volatility mean
xi = .02    # volatility of variance 
corrSV = .1 # correlation between the stock and its variance
corrVT = .0 # correlation between vol and the bond
corrVX = .3 # correlation between vol and the X
lam = .1    # market price of risk for volatility linked contracts
beta0 = volS * corrST - volX * corrSX
beta1 = xi* (volT * corrVT - volX * corrVX - lam)

Nsteps = 100
def simStock():
    dt = T / Nsteps
    WS = 0
    S = 1.
    v = v0
    
    for i in range(Nsteps):
        # generate two correlated normals
        z0 = np.random.normal()
        z1 = corrSV * z0 + np.sqrt(1 - corrSV**2) * np.random.normal()
        
        # generate correlated brownian increments
        dWS = np.sqrt(dt) * z0
        dWV = np.sqrt(dt) * z1
        WS += dWS
        
        # update stock process
        drift = rf + np.sqrt(v) * beta0
        S += S * (drift * dt + np.sqrt(v) * dWS)
        
        # update variance
        dv = k * (v0 - v + beta1 * np.sqrt(v)) * dt + xi * np.sqrt(v) * dWV
        while v + dv < 0.:
            dv = k * (v0 - v + beta1 * np.sqrt(v)) * dt + xi * np.sqrt(v) * np.sqrt(dt) * np.random.normal()
        v += dv
        
    return S, WS

T = 20
delta_T = 2
Nsim = 1000
val = .0
for i in range(Nsim):
    # generate the stock price
    drift = rf + volS*volX*corrSX + volS*volT*corrST
    stock_ratio, WT = simStock()
    
    # generate the LIBOR
    z0 = WT / np.sqrt(T)
    z1 = corrSL * z0 + np.sqrt(1 - corrSL**2) * np.random.normal()
    libor_ratio = np.exp((-0.5 * volL **2) * (T-delta_T) + volL * np.sqrt(T-delta_T) * z1)
    
    val += max((stock_ratio - stock_strike) * (libor_ratio - libor_strike), 0.)

print("Option value: {:.2f}".format(p_0_T * val / Nsim))

Option value: 0.44


We see that under stochastic volatility, the contract has a higher fair value which is consistent with economic intuition.

# Section 2: Libor with Hull-White model

In [20]:
## simulation function for stock price

def sim_S(S0, r, q, cov, s, T, n, dW):
    """
       S0:   initial stock price
       r:    return rate of the stock
       q:    dividend yield
       cov:  covariance between stock price and exchange rate
       s:    volatility of the stock price
       T:    time at the maturity
       n:    number of simulation peirod
       dW:   array of numbers sample from normal distribution
       return:
       S:    sock price at maturity
    """
    
    S = S0
    dt = T/n
    sqdt = np.sqrt(dt)
    
    for i in range(n):
        delta = (r-q-cov)*dt + s*sqdt*dW[i]
        dS = delta*S
        S = S +dS
    
    return S

according to the Hull-White model, the interest rate is:

$$ dr(t) = (\theta(t) -ar)dt +\sigma_rdZ^{Q^d} $$

$\theta(t)$ is fitted from the observed bond price $p(0,t)$ \
$ar$ and $\sigma_r$ is the mean reversion and volatility

By the definition of Libor, we have:

$$L( T-\bigtriangleup , T-\bigtriangleup, T) = \frac{p(T-\bigtriangleup, T-\bigtriangleup) - p(T-\bigtriangleup,T)}{\bigtriangleup p(T-\bigtriangleup,T)}$$

and Hull-White model has closed form for p(t,T), which is:

$$p(t,T) = \frac{p(0,T)}{p(0,t)}e^{B(t,T)f(0,t)-\frac{\sigma^2}{4a}B^2(t,T)(1-e^{-2at})-B(t,T)r(t)}$$

$$B(t,T) = \frac{1}{a}(1-e^{-a(T-t)})$$

$p(0,T)$, $p(0,t)$, and $f(0,t)$ are observable at the beginning 

Therefore, we can estimate Libor rate

## Note:

We will simulate the stock price and rate at the same time, so we need to assume there is a correlation between the interest rate and the stock price, $rho$. The $dZ^{Q^d}$ and $dW^{Q^d}$ need to have a correlation of $rho$

## Code:

In [21]:
## assume a function for the bond price, which is used in Hull-White model 
def p(t):
    return 1/(0.02*np.sqrt(t)+1)

## derive the function for instantaneous forward rate f(0,t) based on p(t) we just assume
def f(t):
    return 0.5/((0.015*np.sqrt(t)+1)*t)

## simulation function for r(t)

def sim_r(a, s, T, dz, r0, n):
    """
       r0: starting value for r(0)
       theta_t: value fitted rate curve at time t
       a: coefficient for r
       s: volatility of r
       T: final time
       dz: sampled from normal distribution
       return:
       r_t: r(T)
    """
    dt = T/n
    r = r0
    t = 0
    sqdt = np.sqrt(dt)
    for i in range(n):
        dr = (p(t) - a*r)*dt + s*sqdt*dz[i]
        r += dr
        t += dt
    return r


## simulation for p(t,T)

def sim_p_t_T(p_T, p_t, f_t, a, s, r_t, T, t):
    """
       p_T: value of p(0,T)
       p_t: value of p(0,t)
       f_t: value of f(0,t)
       a: coefficient of mean reversion
       s: sigma of the rate
       r_t: r(t) estimated by Hull-White model
       T: time at maturity
       t: T-delta
       return:
       p_t_T: value of p(t,T)
    """
    
    B = 1/a*(1-np.exp(-a*(T-t)))
    p_t_T = p_T/p_t*np.exp(B*f_t-s**2/(4*a)*B**2*(1-np.exp(-2*a*t))-B*r_t)
    return p_t_T


##
def sim_dwdz(n, rho):
    """
       generate two arrays dW and dZ, where they are normally distributed and have a correlation of rho
       
       n = length of the array
       rho = value of correlation
    """
    dW = np.zeros(n)
    dZ = np.zeros(n)
    for i in range(n):
        dW[i] = np.random.normal()
        dZ[i] = rho * dW[i] + np.sqrt(1 - rho**2) * np.random.normal()
    return dW, dZ

## simulation for option price 

# stock info
rf = .01     # risk-free rate
volS = .05   # volatility of S
volX = .05   # volatility of X 
volL = .05   # volatility of L(t,T)
corrSX = 0.0  # correlation of S, X
S0 = 1.

# hull-white parameters
a = 1
s = 0.05
r0 = 10

# option parameters
stock_strike = 2.   # k
libor_strike = 1.5  # k'
T =20      # maturity
delta_T = 2 #

n = 100 # number of trials
N = 10000 # nunmber of simulation

val = 0
for i in range(N):
    
    
    t = T-delta_T
    dW,dZ = sim_dwdz(n, corrSX)
    S = sim_S(S0, rf, 0, corrSX*volS*volX, volS, T, n, dW)
    
    r_t = sim_r(a, s, t, dZ, r0, n)
    p_t_T = sim_p_t_T(p(T), p(t), f(t), a, s, r_t, T, t)
    
    libor = (1-p_t_T)/(delta_T*p_t_T)
    
    initial_libor = (p(t)-p(T))/(delta_T*p(T))
    
    libor_ratio = libor/initial_libor
    
    val += max((S - stock_strike) * (libor_ratio - libor_strike), 0.)


print("option value", p(T)*val/N)

option value 0.44000423729715177


0.9179004847782343

# Section 3: Ho-Lee Interest Rate Model
Ho-Lee short rate model has the following format:
$$dr=\theta(t)dt+\sigma dW(t)^Q$$

We import the bond yield to maturity from FRED and forward rates from quandl

In [22]:
import numpy as np
import pandas as pd
import quandl
import matplotlib.pyplot as plt

In [23]:
rf = .01     # risk-free rate
volS = .05   # volatility of S
volX = .05   # volatility of X 
volL = .05   # volatility of L(t,T)
corrSX = 0.0  # correlation of S, X
corrST = 0.0  # correlation of S, p(t,T)
corrSL = 0.3  # correlation of S, L
p_0_T = 0.90  # p(0, T)
volT = 0

# option parameters
stock_strike = 2.   # k
libor_strike = 1.5  # k'
T = 20      # maturity
delta_T = 2 #

In [24]:
def GetFREDMatrix(seriesnames,progress=False,startdate=None,enddate=None):
    import pandas as pd
    import numpy as np
    import fredapi
    fred = fredapi.Fred(api_key='fd97b1fdb076ff1a86aff9b38d7a0e70')

    #Get each time series and load it into a common dataframe
    initialize=True
    for sn in seriesnames:
        if progress: print('Processing ',sn)
        fs=fred.get_series(sn,observation_start=startdate, \
                           observation_end=enddate)
        fs=fs.rename(sn)   #put the name on the column
        if initialize:
            #Set up the dataframe with the first series
            df=pd.DataFrame(fs)
            initialize=False
        else:
            #concatenate the next series to the dataframe
            df=pd.concat([df,fs],axis=1)
    
    #The dataframe has aligned the dates
    #strip out date series
    cdates=df.index.strftime('%Y-%m-%d').tolist()
    ratematrix=[list(df.iloc[i]) for i in range(len(df))]
    return(cdates,ratematrix)
#Done with GetFREDMatrix

In [25]:
seriesnames=['DGS1MO','DGS3MO','DGS6MO','DGS1',
             'DGS2','DGS3','DGS5','DGS7',
             'DGS10','DGS20','DGS30']
cdates,ratematrix=GetFREDMatrix(seriesnames)
hist_forward_rates=quandl.get("FED/SVENF")
sigma_r=0.05

We use the most recent bond data and perform a linear interpolation
$$\frac{y-y_0}{x-x_0}=\frac{y_1-y_0}{x_1-x_0}$$
between the yield rate for the observable maturities.
We then price the bonds using
$$p(0,T)=\frac{1}{(y(0,T)+1)^T}$$
where $y(0,T)$ are the interpolated yield to maturities.

In [26]:
ytm=ratematrix[-1]
maturities=[1/12,1/4,1/2,1,2,3,5,7,10,20,30]
interp_yield=[ytm[0]]
for i in range(len(maturities)-1):
    interp_yield=interp_yield+list(np.linspace(ytm[i],ytm[i+1],int((maturities[i+1]-maturities[i])*12+1)))[1:]
interp_maturities=np.linspace(1/12,30,360)
bond_price=1/(np.array(interp_yield)/100+1)**interp_maturities
bond_price=np.insert(bond_price,0,1)
#bond price at t=0

We use the most recent forward rate data and perform a linear interpolation
$$\frac{y-y_0}{x-x_0}=\frac{y_1-y_0}{x_1-x_0}$$
between the forward rates for the observable maturities.

In [27]:
forward_rate=hist_forward_rates.iloc[-1].values
forward_rate=np.insert(forward_rate,0,0.08) / 100
forward_maturities=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
interp_rate=[forward_rate[0]]
for i in range(len(forward_maturities)-1):
    interp_rate=interp_rate+list(np.linspace(forward_rate[i],forward_rate[i+1],13))[1:]
#instantaneous forward rate at t=0

We use first order difference
$$\frac{\delta f^*}{\delta T}(0,T)\approx\frac{f(0,T)-f(0,T-\delta t)}{\delta t}$$ and $$\frac{\delta f^*}{\delta T}(0,T)\approx\frac{f(0,T+\delta t)-f(0,T)}{\delta t}$$
to approximate derivatives at the end points and use second order central difference
$$\frac{\delta f^*}{\delta T}(0,T)\approx\frac{f(0,T+\delta t)-f(0,T-\delta t)}{2\delta t}$$
to approximate derivatives for the interior points

In [28]:
front=(interp_rate[1]-interp_rate[0])*12
back=(interp_rate[-1]-interp_rate[-2])*12
inst_forward_rate_deriv=(np.array(interp_rate[2:])-np.array(interp_rate[:-2]))*6
inst_forward_rate_deriv=np.insert(inst_forward_rate_deriv,0,front)
inst_forward_rate_deriv=np.insert(inst_forward_rate_deriv,-1,back)
# derivative of instantaneous forward rate at=0 w.r.t to maturity date 

we know that for the Ho-Lee short rate model,
$$\theta(t)=\frac{\delta f^*}{\delta T}(0,t)+\sigma^2t$$
then the Ho-Lee short rate model becomes
$$dr=(\frac{\delta f^*}{\delta T}(0,t)+\sigma^2t)dt+\sigma dW(t)^Q$$
and we have
$$r=r_0+f^*(0,t)+\frac{\sigma^2t^2}{2}+\sigma W(t)^Q$$
assuming $r_0=0$, we have
$$r=f^*(0,t)+\frac{\sigma^2t^2}{2}+\sigma W(t)^Q$$

In [29]:
def r(t,rand):
#Ho-Lee short rate model
#t is time in unit of years
    r=interp_rate[int(t*12)]+(sigma_r*t)**2/2+sigma_r*np.sqrt(t)*rand
    return r

Using the affine term structure, we can derive the closed form of Ho-Lee model bond price as
$$p(t,T)=\frac{p^*(0,T)}{p^*(0,t)}exp\big\{{(T-t)f^*(0,t)-\frac{\sigma^2}{2}t(T-t)^2-(T-t)r(t)}\big\}$$

In [30]:
def p(t,T,rand):
#Ho-Lee term structure bond prices
#t and T are units of year
    p=bond_price[int(T*12)]/bond_price[int(t*12)]*np.exp((T-t)*interp_rate[int(t*12)]-sigma_r**2/2*t*(T-t)**2-(T-t)*r(t,rand))
    return p

by Definition, the LIBOR forward rate is
$$L(t,S,T)=-\frac{p(t,T)-p(t,S)}{(T-S)p(t,T)}$$

In [31]:
def L(t,S,T,rand):
#Libor forward rate from bond price
    l=-(p(t,T,rand)-p(t,S,rand))/((T-S)*p(t,T,rand))
    return l

In [32]:
N = 100000

val = .0
for i in range(N):
    # generate two correlated normals
    z0 = np.random.normal()
    z1 = corrSL * z0 + np.sqrt(1 - corrSL**2) * np.random.normal()
    
    # generate the stock price
    drift = rf + volS*volX*corrSX + volS*volT*corrST
    stock_ratio = np.exp((drift -  0.5 * volS**2 ) * T + volS * np.sqrt(T) * z0)
    
    # generate the LIBOR
    libor_ratio = L(T-delta_T,T-delta_T,T,z1) / L(0,T-delta_T,T,z1)

    val += max((stock_ratio - stock_strike) * (libor_ratio - libor_strike), 0.)

print("Option value: {:.2f}".format(bond_price[int(T*12)] * val / N))

Option value: 0.13


### Conclusion:

Different models led to different option prices because they rely on different assumptions and dynamics. The lognormal libor and stock price model with constant and stochastic volatility yielded close option prices with stochastic volatility leading to a higher price.